# 📚 Tecton in 10 minutes

In this quickstart, you will:

1. Explore a feature from the Feature Registry
2. Generate training data using a FeatureService
3. Serve features for online prediction through a productionized REST API

### 0. Attach the notebook to the cluster and run all commands in the notebook
1. In the notebook menu bar, select **<img src="http://docs.databricks.com/_static/images/notebooks/detached.png"/></a> > `notebook-demo-{cluster-name}-cluster`.
2. When the cluster changes from <img src="http://docs.databricks.com/_static/images/clusters/cluster-starting.png"/></a> to <img src="http://docs.databricks.com/_static/images/clusters/cluster-running.png"/></a>, click **<img src="http://docs.databricks.com/_static/images/notebooks/run-all.png"/></a> Run All**.

In [3]:
# Import Tecton
import tecton
tecton.version.summary()

Version: 0.0.20
Git Commit: 91258309f47b023e6dfadc542cd83773923ccb23
Build Datetime: 2020-10-17T06:33:29

### 1. Explore features from the Feature Registry

🔑 **Concept: Feature Registry**

The Feature Registry includes all of your organization's registered features. Once a feature is registered with Tecton, Tecton manages the orchestration, storage, and serving of that feature. You can review overviews of all of your features' properties by clicking on the Features button in the Tecton Web UI. You can also access all of your registered features in Notebooks using the Tecton SDK.

The cells below do two things:
1. Loads a individual feature from the FeatureService used in this tutorial and reviews the description of that feature
2. Calls a preview of the feature to review a sample dataset

Note: In the unlikely case you received an error running these commands, it can be fixed by loading your Feature Store with the tutorial data [using these instructions](https://staging-docs.tecton.ai/trial/tutorial/environment-set-up.html).

In [5]:
# Load the FeaturePackage ad_ground_truth_ctr_performance_7_days 
fp = tecton.get_feature_package('ad_ground_truth_ctr_performance_7_days')

fp.description

Out[2]: '[SQL Feature] The aggregate CTR of a partner website (clicks / total impressions) over the last 7 days'

In [6]:
# View a preview of feature data
fp.preview()

,ad_id,timestamp,ad_total_clicks_7days,ad_total_impressions_7days
0,3106,2020-10-09,47,813
1,3061,2020-10-09,40,819
2,5934,2020-10-09,37,797
3,3199,2020-10-09,39,704
4,4126,2020-10-09,40,759
5,3280,2020-10-09,47,744
6,5241,2020-10-09,37,767
7,4371,2020-10-09,42,735
8,3800,2020-10-09,52,839
9,5759,2020-10-09,35,815


⚠️ When determining which existing features to use for your model, the Tecton Web UI can help answer important questions, including:
- Who created the feature, and when?
- What is the source data for this feature - is it from batch, streaming, or request-time data?
- What is the logic for the transformation?
- What are some summary statistics associated with this feature?

Once you've completed this quickstart, we recommend you review the Tecton Web UI to get a sense of what information is available there.

📢 **Three things to notice:** 
* Some feature logic may represent multiple features - for example, you can see there are two features generated for ``'ad_ground_truth_ctr_performance_7_days'``. Tecton packages multiple features that are computed together into objects named FeaturePackages.
* Using Tecton to register and manage your features allows for FeaturePackages to be reused and shared across your organization. 
* FeaturePackages have distinct names and be accessed via common methods. This allows for features to be referred to and used as objects of code.

### 2. Generate training data using a FeatureService

🔑 **Concept: FeatureServices**

A FeatureService is a set of features that have been grouped together for use in training and serving. Typically, each deployed model will have one FeatureService to serve features. One Tecton FeatureService can be used  for both generating training data sets and for serving feature values for batch and real-time predictions. Let's see how we can use a FeatureService to generate training data.

The cells below do two things:
1. *Load a set of sample events* - Sample keys for a training set, the timestamps for prediction, and the labels.
2. *Pass those events to a FeatureService* - Tecton augments the sample events to create a training set with new columns for the FeatureService features and their values

Note: These queries may take a few minutes to run.

In [10]:
# Load a set of sample events from your Tecton environment
# NOTE: Although this data is pre-installed for the tutorial, it is more common for a data scientist to collect these events when model building 
events = tecton.get_virtual_data_source('sample_events_for_model')
events.preview()

,ad_id,user_uuid,timestamp,clicked
0,4870,5fa2f8c2-fb23-54cf-a254-c35e9ba8e619,2020-08-04 18:11:17,1
1,5493,7bb7e51f-b9ab-5c22-89cb-c94e90da6f01,2020-08-04 18:11:16,0
2,3014,fc3e2b7c-4b50-5a1b-8221-a49d61a5aa9a,2020-08-04 18:11:14,0
3,5523,85bde9d2-e8bf-523d-a9f6-92bd70318aa5,2020-08-04 18:11:14,0
4,3471,5cd37c53-21a1-5d9c-8592-98ddfed6092e,2020-08-04 18:11:14,0
5,3641,f3d79796-8029-5754-a1dd-fdbeaff5dc6c,2020-08-04 18:11:13,0
6,4656,90c0be00-6dba-585d-84d7-75e266f2e4ec,2020-08-04 18:11:13,1
7,3420,d66c4692-9b21-5bd6-8044-449c0252fb36,2020-08-04 18:11:12,0
8,4338,e3ed709a-d896-54eb-a36a-7ff873e8bda0,2020-08-04 18:11:11,0
9,5504,84a6ce89-bca9-56a2-b01a-e5e9baabf83a,2020-08-04 18:11:11,0


In [11]:
# Generate training data using a Tecton FeatureService: 
fs = tecton.get_feature_service('ctr_prediction_service')
training_data = fs.get_feature_dataframe(events.dataframe())
display(training_data.to_spark())

user_uuid,ad_id,timestamp,clicked,ad_ground_truth_ctr_performance_7_days.ad_total_clicks_7days,ad_ground_truth_ctr_performance_7_days.ad_total_impressions_7days,user_total_ad_frequency_counts.impression_count_1h_1h,user_total_ad_frequency_counts.impression_count_12h_1h,user_total_ad_frequency_counts.impression_count_24h_1h,user_total_ad_frequency_counts.impression_count_72h_1h,user_total_ad_frequency_counts.impression_count_168h_1h,user_ad_impression_counts.impression_count_1h_1h,user_ad_impression_counts.impression_count_12h_1h,user_ad_impression_counts.impression_count_24h_1h,user_ad_impression_counts.impression_count_72h_1h,user_ad_impression_counts.impression_count_168h_1h
0455fce0-dd62-53cb-9d37-035a905964f8,5243,2020-08-04T16:54:12.000+0000,0,17,387,2,12,20,65,154,0,0,0,0,0
1df02b80-2468-57bc-bdab-46af368c18d5,3014,2020-08-04T16:51:51.000+0000,0,14,361,0,10,18,49,132,0,0,0,0,0
45eae98e-281c-5ee6-9d85-8d582cf02232,5021,2020-08-04T16:46:26.000+0000,0,19,443,0,14,24,70,164,0,0,0,0,0
47812ce6-aa17-53f7-864f-cae7329e036a,3967,2020-08-04T16:55:13.000+0000,0,14,404,1,11,23,72,160,0,0,0,0,0
4ff1e670-739e-5c24-9da6-059706a10944,4437,2020-08-04T16:52:42.000+0000,0,20,379,2,12,25,70,149,0,0,0,0,0
55b4d707-4042-536b-99de-7978430ebe23,5435,2020-08-04T16:56:31.000+0000,0,14,381,2,15,26,72,149,0,0,0,0,0
6bccdf8b-3a60-5e5b-a7d5-1483eeef943b,4237,2020-08-04T16:59:47.000+0000,0,17,383,0,16,26,64,130,0,0,0,0,0
9940464a-5e42-5576-8734-a6bc7d62de60,3308,2020-08-04T16:52:58.000+0000,0,16,383,0,6,30,77,165,0,0,0,0,0
9b6c8085-4727-5929-8d04-5792904fdd2e,5838,2020-08-04T16:47:29.000+0000,0,25,411,3,16,28,75,155,0,0,0,0,0
da23fa66-8869-58a1-a21f-9e4c61943e27,5833,2020-08-04T16:45:12.000+0000,0,25,389,1,6,12,63,140,0,0,0,0,0


You can learn more about the FeatureService by reviewing the Tecton Web UI - for example, the features in the FeatureService and operational metrics associated with the API endpoint.


📢 **Three things to notice:** 
* Tecton manages the time dimension of your features for you. That is, the system automatically makes sure that all feature values are accurate with respect to the prediction timestamp and there is no target leakage. 
* This training set is created from a combination of features using batch and streaming data - traditionally a difficult undertaking. You can learn more about the different features in this FeatureService in the Tecton Web UI. 
* Training set generation was done with two lines of code. It is quick to begin building your models, defining your model metrics, and iterating on different algorithms when it is easy to access the features.

### 3. Serve features for online prediction through a productionized REST API

Let's assume the model created from the above training set was put into production. To use that model to make a prediction, it needs the up-to-date data at the `ad_id` and `user_uuid` level (the index of the training set). If you make a request call to Tecton with the `ad_id` and `user_uuid` values you need data for, Tecton will return the feature values in a JSON object.

In [14]:
from pprint import pprint

keys = {
  'user_uuid': '0455fce0-dd62-53cb-9d37-035a905964f8',
  'ad_id': '5243'
}

response = fs.get_feature_vector(keys).to_dict()
pprint(response)

{'ad_ground_truth_ctr_performance_7_days.ad_total_clicks_7days': 25,
 'ad_ground_truth_ctr_performance_7_days.ad_total_impressions_7days': 702,
 'user_ad_impression_counts.impression_count_12h_1h': 0,
 'user_ad_impression_counts.impression_count_168h_1h': 0,
 'user_ad_impression_counts.impression_count_1h_1h': 0,
 'user_ad_impression_counts.impression_count_24h_1h': 0,
 'user_ad_impression_counts.impression_count_72h_1h': 0,
 'user_total_ad_frequency_counts.impression_count_12h_1h': 14,
 'user_total_ad_frequency_counts.impression_count_168h_1h': 269,
 'user_total_ad_frequency_counts.impression_count_1h_1h': 0,
 'user_total_ad_frequency_counts.impression_count_24h_1h': 23,
 'user_total_ad_frequency_counts.impression_count_72h_1h': 104}

The dictionary above is a feature vector - all of the feature values are valid as of this moment in time. To see a different feature vector, you can choose other `ad_id` and `user_uuid` pairs by copying values from the training set above.

📢 **Three things to notice:** 
* These feature values are different than the values in the training set (Section 2 of this notebook). The values in the training set are associated with a specific set of timestamps for training; the feature vector values are true as of the current time.
* These prediction values are accessible as soon as a data scientist defines a FeatureService. The infrastructure for serving in production is managed for you.
* These values are served with production SLAs in mind - Tecton offers a p99 request latency of 100ms.

## Conclusion

In this walkthrough we:
1. Covered key Tecton concepts, including FeatureServices, FeaturePackages, and the Feature Registry
2. Explored features from the Feature Registry
3. Generated training data using a FeatureService
4. Served features at prediction through a productionized API

## What's next?
* Log into the Tecton Web UI using the link provided in your Welcome email. The documentation tutorial section has [a page on Navigating the Tecton Web UI](https://docs.tecton.ai/trial/tutorial/02-navigating-the-ui.html).
* Begin [Taking the Next Steps](https://docs.tecton.ai/trial/tutorial/03-environment-set-up.html)! That's the extended part of our tutorial - where you create features and add new services.

### Tecton Documentation
- <a href="https://docs.tecton.ai" target="_blank">Tecton Docs</a>
- <a href="https://s3-us-west-2.amazonaws.com/tecton.ai.public/documentation/tecton-py/index.html" target="_blank">Tecton Python SDK Docs</a>